In [4]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg
import time
import random
import sys

# Criar sessão do Spark sem Hive
spark = SparkSession.builder \
    .appName("PySpark_Alunos_DB") \
    .config("spark.sql.warehouse.dir", "file:///tmp/spark-warehouse") \
    .getOrCreate()

# Criar dados para alunos
dados_alunos = pd.DataFrame({
    "id_aluno": [1, 2, 3, 4, 5, 6],
    "nome": ["João Silva", "Maria Santos", "Carlos Nunes", "Ana Costa", "Rui Pereira", "Sara Mendes"],
    "idade": [16, 17, 15, 16, 18, 17],
    "turma": [1, 2, 3, 1, 2, 4],
    "desempenho": ["Bom", "Excelente", "Médio", "Bom", "Fraco", "Excelente"]
})

df_alunos = spark.createDataFrame(dados_alunos)

# 🔎 CONSULTAS
print("1️⃣ Listar todos os alunos:")
df_alunos.show()

print("2️⃣ Contar quantos alunos existem:")
df_alunos.count()

print("3️⃣ Calcular a idade média dos alunos:")
df_alunos.select(avg("idade").alias("idade_media")).show()

print("4️⃣ Contar quantos alunos existem por turma:")
df_alunos.groupBy("turma").count().show()

print("5️⃣ Listar alunos com desempenho 'Excelente':")
df_alunos.filter(col("desempenho") == "Excelente").show()

# ============================
# 🔥 POR QUE PYSpark É MELHOR?
# ============================

# 📌 1️⃣ Criar dataset grande (100.000 alunos)
dados_alunos_grande = pd.DataFrame({
    "id_aluno": list(range(1, 100001)),
    "nome": [f"Aluno {i}" for i in range(1, 100001)],
    "idade": [random.randint(15, 20) for _ in range(100000)],
    "turma": [random.randint(1, 4) for _ in range(100000)],
    "desempenho": [random.choice(["Bom", "Excelente", "Médio", "Fraco"]) for _ in range(100000)]
})

df_alunos_grande_pandas = pd.DataFrame(dados_alunos_grande)
df_alunos_grande_pyspark = spark.createDataFrame(dados_alunos_grande)

# 📌 2️⃣ Comparação de tempo entre Pandas e PySpark

# Tempo de execução no Pandas
start = time.time()
df_alunos_grande_pandas.groupby("turma")["idade"].mean()
end = time.time()
print(f"⏳ Tempo com Pandas: {end - start:.4f} segundos")

# Tempo de execução no PySpark
start = time.time()
df_alunos_grande_pyspark.groupBy("turma").agg(avg("idade")).show()
end = time.time()
print(f"🚀 Tempo com PySpark: {end - start:.4f} segundos")

# 📌 3️⃣ Uso de memória
print(f"📌 Uso de memória (Pandas): {sys.getsizeof(df_alunos_grande_pandas)} bytes")
print(f"📌 Uso de memória (PySpark): Não depende da RAM local, processa em cluster!")

# 📌 4️⃣ Mostrar como PySpark pode escalar
print("🔥 PySpark pode processar milhões de registos distribuídos em múltiplos nós!")

# Fechar sessão
spark.stop()


1️⃣ Listar todos os alunos:
+--------+------------+-----+-----+----------+
|id_aluno|        nome|idade|turma|desempenho|
+--------+------------+-----+-----+----------+
|       1|  João Silva|   16|    1|       Bom|
|       2|Maria Santos|   17|    2| Excelente|
|       3|Carlos Nunes|   15|    3|     Médio|
|       4|   Ana Costa|   16|    1|       Bom|
|       5| Rui Pereira|   18|    2|     Fraco|
|       6| Sara Mendes|   17|    4| Excelente|
+--------+------------+-----+-----+----------+

2️⃣ Contar quantos alunos existem:
3️⃣ Calcular a idade média dos alunos:
+-----------+
|idade_media|
+-----------+
|       16.5|
+-----------+

4️⃣ Contar quantos alunos existem por turma:
+-----+-----+
|turma|count|
+-----+-----+
|    1|    2|
|    2|    2|
|    3|    1|
|    4|    1|
+-----+-----+

5️⃣ Listar alunos com desempenho 'Excelente':
+--------+------------+-----+-----+----------+
|id_aluno|        nome|idade|turma|desempenho|
+--------+------------+-----+-----+----------+
|       2|M